In [1]:
!del /Q utils\__pycache__

In [2]:
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
from utils.datasets import GhlKasperskyDataset, TepHarvardDataset, TepKasperskyDataset, SwatItrustDataset
from utils.custom_plots import plot_stacked
from utils.metrics import time_span_metrics
from utils.preserves import save_object, load_object
from utils.watchmen import LimitWatchman, LimitPcaWatchman, SpePcaWatchman, IsolatingWatchman, LinearPredictWatchman

In [4]:
SEED = 1729

# Preparing datasets

In [5]:
datasets = {
    0: GhlKasperskyDataset(),
    1: TepHarvardDataset(),
    2: TepKasperskyDataset(),
    3: SwatItrustDataset(),
}

In [6]:
shake_kwargs = {
    'random_state': SEED,
    'valid_test_ratio': 0.3,
}
for d in datasets:
    datasets[d].shake_not_stir(**shake_kwargs)

# Preparing watchmen

In [7]:
watchmen = {
    0: {  # GhlKasperskyDataset
        0: LimitWatchman(),
        1: LimitPcaWatchman(n_components=3),
        2: SpePcaWatchman(n_components=3),
        3: IsolatingWatchman(random_state=SEED),
        4: LinearPredictWatchman(random_state=SEED),
    },
    1: {  # TepHarvardDataset
        0: LimitWatchman(),  # better: ewma='3 min'
        1: LimitPcaWatchman(n_components=3),  # better: n_components=12
        2: SpePcaWatchman(n_components=3),  # better: n_components=12
        3: IsolatingWatchman(random_state=SEED),
        4: LinearPredictWatchman(random_state=SEED),
    },
    2: {  # TepKasperskyDataset
        0: LimitWatchman(),
        1: LimitPcaWatchman(n_components=3),
        2: SpePcaWatchman(n_components=3),
        3: IsolatingWatchman(random_state=SEED),  # better: max_samples=1024
        4: LinearPredictWatchman(random_state=SEED),
    },
    3: {  # SwatItrustDataset
        0: LimitWatchman(),
        1: LimitPcaWatchman(n_components=3),
        2: SpePcaWatchman(n_components=3),
        3: IsolatingWatchman(random_state=SEED),
        4: LinearPredictWatchman(random_state=SEED),
    },
}

# Learning

## Prefit

In [8]:
for d in datasets:
    for data, _, _ in tqdm(datasets[d].train_generator(), desc=f'Prefit on train {d}'):
        for w in watchmen[d]:
            watchmen[d][w].prefit(data)

Prefit on train 0: 0it [00:00, ?it/s]

Prefit on train 1: 0it [00:00, ?it/s]

Prefit on train 2: 0it [00:00, ?it/s]

Prefit on train 3: 0it [00:00, ?it/s]

## Fit

In [9]:
for d in datasets:
    for data, _, _ in tqdm(datasets[d].train_generator(), desc=f'Fit on train {d}'):
        for w in watchmen[d]:
            watchmen[d][w].partial_fit(data)

Fit on train 0: 0it [00:00, ?it/s]

Fit on train 1: 0it [00:00, ?it/s]

Fit on train 2: 0it [00:00, ?it/s]

Fit on train 3: 0it [00:00, ?it/s]

## Save

In [10]:
for d in datasets:
    for w in watchmen[d]:
        save_object(watchmen[d][w], f'{datasets[d].__class__.__name__}-watchman_{w}')

# Self-checking
Check watchmen on train. Theoretically all metrics must be NaN.

## Load

In [11]:
for d in datasets:
    for w in watchmen[d]:
        watchmen[d][w] = load_object(f'{datasets[d].__class__.__name__}-watchman_{w}')

## Throw stones

In [12]:
stones = dict()
for d in datasets:
    stones[d] = dict()
    for w in watchmen[d]:
        stones[d][w] = list()

In [13]:
for d in datasets:
    for data, faults, info in tqdm(datasets[d].train_generator(), desc=f'Detect on train {d}'):
        for w in watchmen[d]:
            detect = watchmen[d][w].predict(data)
            stones[d][w].append(pd.concat([faults, detect], axis=1))
            stones[d][w][-1].index.name = info

Detect on train 0: 0it [00:00, ?it/s]

Detect on train 1: 0it [00:00, ?it/s]

Detect on train 2: 0it [00:00, ?it/s]

Detect on train 3: 0it [00:00, ?it/s]

## Checks

In [14]:
metrics = ('precision', 'recall', 'f1_score')
checks = {str(datasets[d]): pd.DataFrame(columns=metrics) for d in datasets}

In [15]:
# alone
for d in datasets:
    for w in tqdm(watchmen[d], desc=f'Collect stones d{d}'):
        exam_paper = pd.DataFrame(columns=metrics)
        for i_st, st in enumerate(stones[d][w]):
            exam_paper.loc[i_st, metrics] = time_span_metrics(st.iloc[:, 0], st.iloc[:, 1:])
        checks[str(datasets[d])].loc[str(watchmen[d][w]), metrics] = exam_paper.mean().values

Collect stones d0:   0%|          | 0/5 [00:00<?, ?it/s]

Collect stones d1:   0%|          | 0/5 [00:00<?, ?it/s]

Collect stones d2:   0%|          | 0/5 [00:00<?, ?it/s]

Collect stones d3:   0%|          | 0/5 [00:00<?, ?it/s]

In [16]:
# ensemble
max_threshold = 5
for d in datasets:
    exam_papers = dict()
    for threshold in range(1, max_threshold+1):
        exam_papers[threshold] = pd.DataFrame(columns=metrics)
    for i_st, st in tqdm(enumerate(stones[d][0]), desc=f'Ensembling stones d{d}'):
        opinions = pd.concat([stones[d][w][i_st].iloc[:, 1:].sum(axis=1) for w in watchmen[d]], axis=1)
        for threshold in range(1, max_threshold+1):
            detect = (opinions.sum(axis=1) >= threshold).astype('uint8')
            exam_papers[threshold].loc[i_st, metrics] = time_span_metrics(st.iloc[:, 0], detect)
    for threshold in range(1, max_threshold+1):
        checks[str(datasets[d])].loc[f'ensemble(threshold={threshold})', metrics] = exam_papers[threshold].mean().values

Ensembling stones d0: 0it [00:00, ?it/s]

Ensembling stones d1: 0it [00:00, ?it/s]

Ensembling stones d2: 0it [00:00, ?it/s]

Ensembling stones d3: 0it [00:00, ?it/s]

In [17]:
for d in checks:
    print(d)
    display(checks[d])

GhlKasperskyDataset(E:\Datasets\GHL)


,precision,recall,f1_score
LimitWatchman(ewma=None),NaN,NaN,NaN
LimitPcaWatchman(n_components=3),NaN,NaN,NaN
SpePcaWatchman(n_components=3),NaN,NaN,NaN
IsolatingWatchman(n_trees=17),0.0,NaN,0.0
LinearPredictWatchman(n_features=12),0.0,NaN,0.0
ensemble(threshold=1),0.0,NaN,0.0
ensemble(threshold=2),0.0,NaN,0.0
ensemble(threshold=3),0.0,NaN,0.0
ensemble(threshold=4),NaN,NaN,NaN
ensemble(threshold=5),NaN,NaN,NaN


TepHarvardDataset(E:\Datasets\TEP\dataverse)


,precision,recall,f1_score
LimitWatchman(ewma=None),NaN,NaN,NaN
LimitPcaWatchman(n_components=3),NaN,NaN,NaN
SpePcaWatchman(n_components=3),NaN,NaN,NaN
IsolatingWatchman(n_trees=500),0.0,NaN,0.0
LinearPredictWatchman(n_features=52),0.0,NaN,0.0
ensemble(threshold=1),0.0,NaN,0.0
ensemble(threshold=2),0.0,NaN,0.0
ensemble(threshold=3),NaN,NaN,NaN
ensemble(threshold=4),NaN,NaN,NaN
ensemble(threshold=5),NaN,NaN,NaN


TepKasperskyDataset(E:\Datasets\TEP\kaspersky)


,precision,recall,f1_score
LimitWatchman(ewma=None),NaN,NaN,NaN
LimitPcaWatchman(n_components=3),NaN,NaN,NaN
SpePcaWatchman(n_components=3),NaN,NaN,NaN
IsolatingWatchman(n_trees=400),0.0,NaN,0.0
LinearPredictWatchman(n_features=53),0.0,NaN,0.0
ensemble(threshold=1),0.0,NaN,0.0
ensemble(threshold=2),0.0,NaN,0.0
ensemble(threshold=3),0.0,NaN,0.0
ensemble(threshold=4),NaN,NaN,NaN
ensemble(threshold=5),NaN,NaN,NaN


SwatItrustDataset(E:\Datasets\SWaT\dataset12)


,precision,recall,f1_score
LimitWatchman(ewma=None),NaN,NaN,NaN
LimitPcaWatchman(n_components=3),NaN,NaN,NaN
SpePcaWatchman(n_components=3),NaN,NaN,NaN
IsolatingWatchman(n_trees=6),0.0,NaN,0.0
LinearPredictWatchman(n_features=51),0.0,NaN,0.0
ensemble(threshold=1),0.0,NaN,0.0
ensemble(threshold=2),0.0,NaN,0.0
ensemble(threshold=3),0.0,NaN,0.0
ensemble(threshold=4),NaN,NaN,NaN
ensemble(threshold=5),NaN,NaN,NaN


IsolatingWatchman and LinearPredictWatchman detecting anomalies on train.

Ensemble with threshold=4+ don't detect anomalies on train. Let's use threshold=4 for reducing false positive results.